In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from my_functions import *
%matplotlib notebook

In [ ]:
pm_data = np.load('npy/pm_eLAE_1deg.npy')
f_name = 'Source_cats/Source_cat_LAE_1deg.npy'
mock = np.load(f_name, allow_pickle=True).item()
errors = np.load('npy/errors5Sigma.npy')[:,1]

In [ ]:
filters_tags = load_filter_tags()
tcurves = load_tcurves(filters_tags)
w_central = np.array(central_wavelength(tcurves))

In [ ]:
bb_fwhm = [
    nb_fwhm(tcurves, -4, True),
    nb_fwhm(tcurves, -3, True),
    nb_fwhm(tcurves, -2, True),
    nb_fwhm(tcurves, -1, True)
]

In [ ]:
def stack_estimation(pm_flx, pm_err, nb_c, N_nb):
    '''
    Returns the weighted average and std of N_nb Narrow Bands
    arround the central one.
    '''
    nb_idx_Arr = np.array([*range(nb_c-N_nb, nb_c), *range(nb_c+1, nb_c+N_nb-1)])
    
    flx = pm_flx[nb_idx_Arr]
    err = pm_err[nb_idx_Arr]
    
    avg = np.average(flx, axis=0, weights=err)
    std = np.average((flx - avg)**2, axis=0, weights=err)**0.5
    
    return avg, std

In [ ]:
nb_c = 12
fcont, cont_err = stack_estimation(pm_data, errors, 12, 10)

In [ ]:
plt.close('all')
for mock_n in range(15):
    pm = pm_data[:,mock_n]
    
    fig, ax = plt.subplots()
    ax.plot( mock['w_Arr'], mock['SEDs'][mock_n], c='orange', lw=2, zorder=-1)
    ax.scatter(w_central[:-3], pm[:-3])
    ax.errorbar(w_central[:-3], pm[:-3], yerr=errors[:-3], fmt='none')
    ax.scatter(w_central[nb_c], pm[nb_c], c='black')

    ax.scatter(w_central[-4], pm[-4], c='purple' , marker='s')
    ax.scatter(w_central[-3], pm[-3], c='green'  , marker='s')
    ax.scatter(w_central[-2], pm[-2], c='red'    , marker='s')
    ax.scatter(w_central[-1], pm[-1], c='dimgray', marker='s')

    ax.errorbar(w_central[-4], pm[-4], xerr=bb_fwhm[-4]/2, fmt='none', color='purple' , elinewidth=4)
    ax.errorbar(w_central[-3], pm[-3], xerr=bb_fwhm[-3]/2, fmt='none', color='green'  , elinewidth=4)
    ax.errorbar(w_central[-2], pm[-2], xerr=bb_fwhm[-2]/2, fmt='none', color='red'    , elinewidth=4)
    ax.errorbar(w_central[-1], pm[-1], xerr=bb_fwhm[-1]/2, fmt='none', color='dimgray', elinewidth=4)

    ax.set_xlabel('$\lambda\ (\AA)$', size=15)
    ax.set_ylabel('$f_\lambda$ (erg cm$^{-2}$ s$^{-1}$ $\AA^{-1}$)', size=15)

    # Fit line
    ax.errorbar(w_central[nb_c], fcont[mock_n], yerr=cont_err[mock_n],
                c='black', marker='s', markersize=8, markerfacecolor='none')

    plt.show()